# Inheritance: For Better or for Worse

In [1]:
class Character:
    def __init__(self, name, **kwargs):
        self.name = name


class Animal(Character):
    def __init__(self, breed, **kwargs):
        self.breed = breed
        super().__init__(**kwargs)

In [2]:
dog = Animal("Corgi", name="Rusty")

In [3]:
print(dog.name, dog.breed)

Rusty Corgi


## Subclassing Built-In Types Is Tricky

In [4]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict(one=1)
dd

{'one': 1}

In [5]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [6]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

1. `DoppelDict.__setitem__` duplicates values when storing (for no good reason, just to have a visible effect). It works by delegating to the superclass.

2. The `__init__` method inherited from dict clearly ignored that `__setitem__` was overridden: the value of 'one' is not duplicated.

3. The [] operator calls our `__setitem__` and works as expected: 'two' maps to the duplicated value [2, 2].

4. The update method from dict does not use our version of `__setitem__` either: the value of 'three' was not duplicated.

This built-in behavior is a violation of a basic rule of object-oriented programming: the search for methods should always start from the class of the receiver (self), even when the call happens inside a method implemented in a superclass. This is what is called "late binding".

The problem is not limited to calls within an instance—whether `self.get()` calls `self.__getitem__()` — but also happens with overridden methods of other classes that should be called by the built-in methods.

In [7]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict(a='foo')
ad

{'a': 42}

In [8]:
d = {}
d.update(ad)
d['a']

'foo'

In [9]:
d

{'a': 'foo'}

1. `AnswerDict.__getitem__` always returns 42, no matter what the key.

2. ad is an `AnswerDict` loaded with the key-value pair ('a', 'foo').

3. `ad['a']` returns 42, as expected.

4. d is an instance of plain `dict`, which we update with `ad`.

5. The `dict.update` method ignored our `AnswerDict.__getitem__`.

### If you subclass collections.UserDict instead of dict, the issues above will be fixedd

In [10]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [11]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [12]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [13]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict2(a='foo')
ad['a']

42

In [14]:
d = {}
d.update(ad)
d['a']

42

In [15]:
d

{'a': 42}

## Multiple Inheritance and Method Resolution Order

In [16]:
class Root:
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'

class A(Root):
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()

class B(Root):
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')

class U:
    def ping(self):
        print(f'{self}.ping() in U')
        # super().ping()

class Leaf(A, B):
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

class LeafUA(U, A):
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

class LeafAUB(A,U,B):
    def ping(self):
        print(f'{self}.ping() in LeafAUB')
        super().ping()

class LeafABU(A,B,U):
    def ping(self):
        print(f'{self}.ping() in LeafAUB')
        super().ping()

In [17]:
print(Leaf.__mro__)
Leaf().ping()
# Both A B inherits Root

(<class '__main__.Leaf'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Root'>, <class 'object'>)
<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [18]:
Leaf().pong()
# Both A B inherits Root but pong() in B doesn't have super() method hence it stops the Root call

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [19]:
LeafUA().ping()
# U doesn't inherit Root but A does. However, since U starts first on the left. MRO will resolve in a way that U is a subclass of A. Hence U's super().ping() will call A

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U


In [20]:
print(LeafAU.__mro__)
LeafAU().ping()
# U doesn't inherit Root but A does. However, MRO will resolve A then the super().ping() from A immediately because U doesn't inherit Root on the same inheritance level as A

(<class '__main__.LeafAU'>, <class '__main__.A'>, <class '__main__.Root'>, <class '__main__.U'>, <class 'object'>)
<instance of LeafAU>.ping() in LeafUA
<instance of LeafAU>.ping() in A
<instance of LeafAU>.ping() in Root


In [21]:
print(LeafAUB.__mro__,"\n", LeafABU.__mro__)

(<class '__main__.LeafAUB'>, <class '__main__.A'>, <class '__main__.U'>, <class '__main__.B'>, <class '__main__.Root'>, <class 'object'>) 
 (<class '__main__.LeafABU'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Root'>, <class '__main__.U'>, <class 'object'>)


## Mixin Classes

A mixin class is designed to be subclassed together with at least one other class in a multiple inheritance arrangement. A mixin is not supposed to be the only base class of a concrete class, because it does not provide all the functionality for a concrete object, but only adds or customizes the behavior of child or sibling classes.

### Example

In [22]:
import collections

def _upper(key):
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

Since every method ot `UpperCaseMixin` calls `super()`, this mixin depends on a sibling class that implements or inherits methods with the same signature. To make its contribution, a mixin usually needs to appear before other classes in the MRO of a subclass that uses it.
In practice, that means mixins must appear first in the tuple of base classes in a class declaration.

In [23]:
class UpperDict(UpperCaseMixin, collections.UserDict):
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):
    """Specialized 'Counter' that uppercases string keys"""

In [24]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
d.keys()

KeysView({'A': 'letter A', 2: 'digit two'})

In [25]:
d['b'] = 'letter B'
'b' in d

True

In [26]:
d['a'], d.get('B')

('letter A', 'letter B')

In [27]:
list(d.keys())

['A', 2, 'B']

In [28]:
# And a quick demonstration of UpperCounter:
c = UpperCounter('BaNanA')
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]

## Favor Object Composition over Class Inheritance
Subclassing is a form of tight coupling, and tall inheritance trees tend to be brittle.

When dealing with multiple inheritance, it’s useful to keep straight the reasons why subclassing is done in each particular case. The main reasons are:

- Inheritance of interface creates a subtype, implying an “is-a” relationship. This is best done with ABCs.
- Inheritance of implementation avoids code duplication by reuse. Mixins can help with this.

Interface inheritance should use only ABCs as base classes, if possible.

### Make Interfaces Explicit with ABCs

In modern Python, if a class is intended to define an interface, it should be an explicit ABC or a `typing.Protocol` subclass

## Use Explicit Mixins for Code Reuse

If a class is designed to provide method implementations for reuse by multiple unrelated subclasses, without implying an “is-a” relationship, it should be an explicit mixin class. Conceptually, a mixin does not define a new type; it merely bundles methods for reuse. A mixin should never be instantiated, and concrete classes should not inherit only from a mixin. Each mixin should provide a single specific behavior, implementing few and very closely related methods. Mixins should avoid keeping any internal state; i.e., a mixin class should not have instance attributes.

## Aggregate Classes to Users

A class that is constructed primarily by inheriting from mixins and does not add its own structure or behavior is called an aggregate class.

If some combination of ABCs or mixins is particularly useful to client code, provide a class that brings them together in a sensible way.

For example, here is the complete source code for the Django `ListView` class

```python
class ListView(MultipleObjectTemplateResponseMixin, BaseListView):
    """
    Render some list of objects, set by `self.model` or `self.queryset`.
    `self.queryset` can actually be any iterable of items, not just a queryset.
    """
```

The body of `ListView` is empty, but the class provides a useful service: it brings together a mixin and a base class that should be used together.

__Note__ that aggregate classes don’t have to be completely empty, but they often are.

## Subclass Only Classes Designed for Subclassing

Subclassing any complex class and overriding its methods is error-prone because the superclass methods may ignore the subclass overrides in unexpected ways. As much as possible, avoid overriding methods, or at least restrain yourself to subclassing classes which are designed to be easily extended, and only in the ways in which they were designed to be extended.

How do we know whether or how a class was designed to be extended? The answer is Documentation.
For example, Python’s `socketserver` package is described as “a framework for network servers.” Its `BaseServer` class is designed for subclassing, as the name suggests.

In Python ≥ 3.8, a new way of making those design constraints explicit is provided by PEP 591—Adding a final qualifier to typing. The PEP introduces a `@final` decorator that can be applied to classes or individual methods, so that IDEs or type checkers can report misguided attempts to subclass those classes or override those methods.

## Avoid Subclassing from Concrete Classes

Subclassing concrete classes is more dangerous than subclassing ABCs and mixins, because instances of concrete classes usually have internal state that can easily be corrupted when you override methods that depend on that state. Even if your methods cooperate by calling `super()`, and the internal state is held in private attributes using the `__x` syntax, there are still countless ways a method override can introduce bugs.

In other words, it is recommended that only abstract classes should be subclassed. If you must use subclassing for code reuse, then the code intended for reuse should be in mixin methods of ABCs or in explicitly named mixin classes.